In [266]:
import numpy as np
import pandas as pd 
import glob, re

In [267]:
#提出用ファイル読み込み
#データはairレジのレストランID+4/23～5/31までの日付とvisitors(予測する値)
test_df = pd.read_csv('sample_submission.csv')
test_df.head(20)


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
5,air_00a91d42b08b08d9_2017-04-28,0
6,air_00a91d42b08b08d9_2017-04-29,0
7,air_00a91d42b08b08d9_2017-04-30,0
8,air_00a91d42b08b08d9_2017-05-01,0
9,air_00a91d42b08b08d9_2017-05-02,0


In [268]:
#idをstore_idとvisit_dateに分離 visitorsをdrop
test_df['store_id'] = test_df['id'].str[:20]
test_df['visit_date'] = test_df['id'].str[21:]

test_df.drop(['visitors'],axis=1,inplace=True)
#日付の型をobjectからdatetimeに変換
test_df['visit_date'] = pd.to_datetime(test_df['visit_date'])

test_df.head()

,id,store_id,visit_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27


In [269]:
air_data = pd.read_csv('air_visit_data.csv',parse_dates=['visit_date'])

air_data.shape


(252108, 3)

In [270]:
air_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [271]:
check_store_sample = air_data[air_data['air_store_id'] == 'air_00a91d42b08b08d9']

check_store_sample.describe()

check_store_sample.visit_date.describe()

count                     232
unique                    232
top       2016-09-08 00:00:00
freq                        1
first     2016-07-01 00:00:00
last      2017-04-22 00:00:00
Name: visit_date, dtype: object

In [272]:
#air_dataの日付（visit_date）を曜日（dow）へ変換
air_data['dow'] = air_data['visit_date'].dt.dayofweek

# air_dataから2017-01-28以降のデータを切り出して訓練データ「train」へ格納
train = air_data[air_data['visit_date'] > '2017-01-28'].reset_index()

train['dow'] = train['visit_date'].dt.dayofweek
test_df['dow'] = test_df['visit_date'].dt.dayofweek

test_df.head()
train.head()

,index,air_store_id,visit_date,visitors,dow
0,313,air_ba937bf13d40fb24,2017-01-29,1,6
1,314,air_ba937bf13d40fb24,2017-01-30,4,0
2,315,air_ba937bf13d40fb24,2017-01-31,12,1
3,316,air_ba937bf13d40fb24,2017-02-01,10,2
4,317,air_ba937bf13d40fb24,2017-02-02,19,3


In [273]:
# trainからair_store_idとdowをグルーピングしてvisitorsの中央値（median）を算出
aggregation = {'visitors' : {'total_visitors' : 'median'}}

agg_data = train.groupby(['air_store_id', 'dow']).agg(aggregation).reset_index()

agg_data.columns = ['air_store_id', 'dow', 'visitors']
agg_data['visitors']=agg_data['visitors']

agg_data.head(12)

C:\Users\inada\Anaconda3\lib\site-packages\pandas\core\groupby.py:3961: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,air_store_id,dow,visitors
0,air_00a91d42b08b08d9,0,24.0
1,air_00a91d42b08b08d9,1,30.5
2,air_00a91d42b08b08d9,2,28.0
3,air_00a91d42b08b08d9,3,33.5
4,air_00a91d42b08b08d9,4,37.0
5,air_00a91d42b08b08d9,5,10.0
6,air_0164b9927d20bcc3,0,5.0
7,air_0164b9927d20bcc3,1,10.5
8,air_0164b9927d20bcc3,2,8.0
9,air_0164b9927d20bcc3,3,11.5


In [274]:
merged = pd.merge(test_df, agg_data, how='left', left_on=['store_id', 'dow'], right_on=['air_store_id', 'dow'])

final = merged[['id','visitors']]

final.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,NaN
1,air_00a91d42b08b08d9_2017-04-24,24.0
2,air_00a91d42b08b08d9_2017-04-25,30.5
3,air_00a91d42b08b08d9_2017-04-26,28.0
4,air_00a91d42b08b08d9_2017-04-27,33.5


In [275]:


# fillna関数を使ってvisitorsのNaNへ0を入れておく
final.fillna(0, inplace=True)

# 念のため確認
missing_values_table(final)


C:\Users\inada\Anaconda3\lib\site-packages\pandas\core\frame.py:2852: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Missing Values,% of Total Values
id,0,0.0
visitors,0,0.0


In [276]:
#全ファイル読み込み
air_store_info = pd.read_csv('air_store_info.csv')
date_info = pd.read_csv('date_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
hpg_reserve = pd.read_csv('hpg_reserve.csv')
air_reserve = pd.read_csv('air_reserve.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')

In [277]:
date_info[date_info['holiday_flg'] == 1].head(10)

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
10,2016-01-11,Monday,1
41,2016-02-11,Thursday,1
79,2016-03-20,Sunday,1
80,2016-03-21,Monday,1
119,2016-04-29,Friday,1
123,2016-05-03,Tuesday,1
124,2016-05-04,Wednesday,1


In [278]:
#date_infoが1かつ土日のものを格納
weekend_hdays = date_info.apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
 
# date_infoの該当の箇所のフラグを1から0へ更新をする
date_info.loc[weekend_hdays, 'holiday_flg'] = 0

In [279]:

# 該当の日付＋1　÷ 全部の日付の個数で重みを計算
# date_info.indexの値が小さい＝より昔のデータ
date_info['weight'] = (date_info.index + 1) / len(date_info) 
 
#ヘッダーとテイルの情報を出して確認
date_info.head()
date_info.tail()

,calendar_date,day_of_week,holiday_flg,weight
512,2017-05-27,Saturday,0,0.992263
513,2017-05-28,Sunday,0,0.994197
514,2017-05-29,Monday,0,0.996132
515,2017-05-30,Tuesday,0,0.998066
516,2017-05-31,Wednesday,0,1.000000


In [280]:

# air_visit_dataと重みを追加したdate_infoをマージさせてvisit_dataを作成
# visit_dataから不必要なcalendar_dateを落とす
visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
 
# visit_dataの実客数にnp.log1pの対数関数を使って処理
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)
 
# visit_dataの確認
visit_data.head(10)

,air_store_id,visit_date,visitors,day_of_week,holiday_flg,weight
0,air_ba937bf13d40fb24,2016-01-13,3.258097,Wednesday,0,0.025145
1,air_ba937bf13d40fb24,2016-01-14,3.496508,Thursday,0,0.027079
2,air_ba937bf13d40fb24,2016-01-15,3.401197,Friday,0,0.029014
3,air_ba937bf13d40fb24,2016-01-16,3.135494,Saturday,0,0.030948
4,air_ba937bf13d40fb24,2016-01-18,1.945910,Monday,0,0.034816
5,air_ba937bf13d40fb24,2016-01-19,2.302585,Tuesday,0,0.036750
6,air_ba937bf13d40fb24,2016-01-20,3.465736,Wednesday,0,0.038685
7,air_ba937bf13d40fb24,2016-01-21,3.091042,Thursday,0,0.040619
8,air_ba937bf13d40fb24,2016-01-22,2.944439,Friday,0,0.042553
9,air_ba937bf13d40fb24,2016-01-23,3.295837,Saturday,0,0.044487


In [281]:

# wmean（重み付き平均）の式を格納
wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
 
 
# グルーピングして重み付き平均を算出
visitors = visit_data.groupby(
    ['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) 
 
# データを確認
visitors.head(10)

,air_store_id,day_of_week,holiday_flg,visitors
0,air_00a91d42b08b08d9,Friday,0,3.590026
1,air_00a91d42b08b08d9,Monday,0,3.091672
2,air_00a91d42b08b08d9,Monday,1,3.091042
3,air_00a91d42b08b08d9,Saturday,0,2.526836
4,air_00a91d42b08b08d9,Sunday,0,1.098612
5,air_00a91d42b08b08d9,Thursday,0,3.414904
6,air_00a91d42b08b08d9,Tuesday,0,3.173611
7,air_00a91d42b08b08d9,Wednesday,0,3.330028
8,air_0164b9927d20bcc3,Friday,0,2.327400
9,air_0164b9927d20bcc3,Friday,1,2.747824


In [282]:


# sample_submissionのIDをレストランIDや日付に分ける
sample_submission['air_store_id'] = sample_submission.id.map(
    lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])

# 重み付き平均で予測したvisitorsとsample_submissionをマージする
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(
    visitors, on=['air_store_id', 'day_of_week', 'holiday_flg'], how='left')

# データセットを確認してみよう
sample_submission.head()


,id,air_store_id,calendar_date,day_of_week,holiday_flg,weight,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.926499,1.098612
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.928433,3.091672
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.930368,3.173611
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.932302,3.330028
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.934236,3.414904


In [283]:
# NaNを探してテーブルにする関数
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 

In [284]:

# sampe_submissionの欠損データを確認
missing_values_table(sample_submission)

,Missing Values,% of Total Values
id,0,0.000000
air_store_id,0,0.000000
calendar_date,0,0.000000
day_of_week,0,0.000000
holiday_flg,0,0.000000
weight,0,0.000000
visitors,668,2.086261


In [285]:
# 「air_store_id」と「 day_of_week」のみで欠損データに重み平均を入れる
missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=(
        'air_store_id', 'day_of_week'), how='left')['visitors_y'].values
 
# 改めて欠損データの確認
missing_values_table(sample_submission)

,Missing Values,% of Total Values
id,0,0.000000
air_store_id,0,0.000000
calendar_date,0,0.000000
day_of_week,0,0.000000
holiday_flg,0,0.000000
weight,0,0.000000
visitors,448,1.399169


In [286]:

# 「air_store_id」のみの重み付き平均を計算して欠損データへ入れる
missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values
 
# 欠損データを確認
missing_values_table(sample_submission)

,Missing Values,% of Total Values
id,0,0.0
air_store_id,0,0.0
calendar_date,0,0.0
day_of_week,0,0.0
holiday_flg,0,0.0
weight,0,0.0
visitors,0,0.0


In [287]:
sample_submission.head()

,id,air_store_id,calendar_date,day_of_week,holiday_flg,weight,visitors
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,Sunday,0,0.926499,1.098612
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,Monday,0,0.928433,3.091672
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,0.930368,3.173611
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,0.932302,3.330028
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,Thursday,0,0.934236,3.414904


In [288]:
# visitorsをnp.expm1で処理して実客数へ戻す
sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
 
# 提出フォーマットの規定に合うように処理してsub_fileへ格納
sample_submission = sample_submission[['id', 'visitors']]
final['visitors'][final['visitors'] ==0] = sample_submission['visitors'][final['visitors'] ==0]
sub_file = final.copy()
 
# データの確認
sub_file.head()

C:\Users\inada\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\inada\Anaconda3\lib\site-packages\pandas\core\generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\inada\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.0
1,air_00a91d42b08b08d9_2017-04-24,24.0
2,air_00a91d42b08b08d9_2017-04-25,30.5
3,air_00a91d42b08b08d9_2017-04-26,28.0
4,air_00a91d42b08b08d9_2017-04-27,33.5


In [289]:

# 算術平均をnp.meanで算出
sub_file['visitors'] = np.mean([final['visitors'], sample_submission['visitors']], axis = 0)
sub_file.to_csv('sub_math_mean_1.csv', index=False)
 
# 相乗平均を算出
sub_file['visitors'] = (final['visitors'] * sample_submission['visitors']) ** (1/2)
sub_file.to_csv('sub_geo_mean_1.csv', index=False)
 
# 調和平均を算出
sub_file['visitors'] = 2/(1/final['visitors'] + 1/sample_submission['visitors'])
sub_file.to_csv('sub_hrm_mean_1.csv', index=False)